In [42]:
import pandas as pd

## **Carga de datos**

In [43]:
clientes_raw = pd.read_excel('../datasets/raw/Clientes.xlsx')
materiales_raw = pd.read_excel('../datasets/raw/Materiales.xlsx')
in_market_raw = pd.read_excel('../datasets/raw/In_Market.xlsx')
to_market_raw = pd.read_excel('../datasets//raw/To_Market.xlsx')

# **ETL**

## **1 . Maestro de Clientes.xlsx**

Columnas sugeridas:
- ID Cliente
- Nombre Cliente
- Segmento / Categoría

Se carga dataframe base con datos crudos.

In [44]:
clientes_raw

,Codigo Cliente,CUSTOMER STD DESC,Customer_Description,CUSTOMER DESC INST,CAT MAESTRO VIEJO,STATUS,Sold To - Ship To,Sold To Code,Ship To Code,Ship To No.,...,MONTH,DAY,DATE,ADDRESS SOLD TO,CP SOLD TO,ADDRESS SHIP TO,CP SHIP TO,CHANNEL,SUB_CHANNEL,GIVE IMS
0,906600,NaN,TRANSPORTISTAS NUTRICIONALES,NaN,NaN,NaN,SOLD TO,906600,-,-,...,6,21,37063,AV. COYOACAN NO. 1622COL. DEL VALLEMEXICO D F,4980,No Address,No C.P.,NaN,NaN,NaN
1,909009,ABBOTT STORE,ABBOTT LABORATORIES DE MEXICO,NaN,NaN,NO LEIDO,SOLD TO,909009,-,-,...,6,22,37064,AV COYOACAN 1622COL DEL VALLEMEXICO D F 03100,4980,No Address,No C.P.,TRADE,Retail,NO
2,400043,NaN,ZAVALA SALAZAR LAURA ELENA,NaN,NaN,NaN,SOLD TO,400043,-,-,...,8,20,37853,BOSQUE DE LOS LEONES NO 231TRIANA SECTOR BOSQU...,66620,No Address,No C.P.,NaN,NaN,NaN
3,400047,NaN,ORTIZ BERNAL ANA DANIELA,NaN,NaN,NaN,SOLD TO,400047,-,-,...,8,20,37853,VOLCAN POPOCATEPETL 4500COL URBANO ZAPOPAN JAL...,45070,No Address,No C.P.,NaN,NaN,NaN
4,400050,NaN,AGUILAR REYES ALEJANDRO,NaN,NaN,NaN,SOLD TO,400050,-,-,...,8,20,37853,CALZ LA VIGA 901 SECC 2 GRUPO5 CASA 70 COL BAR...,8800,No Address,No C.P.,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
333,801850,OTHER GOVERNMENT DISTRIBUTORS,HOSPITAL GENERAL DE MEXICO,"HOSPITAL GENERAL DE MEXICO ""DR. EDUARDO LICEAGA""",ISSSTE,NO LEIDO,SOLD TO,801850,-,1,...,7,24,45862,"DR BALMIS NO 148 COL DOCTORESCUAUHTEMOC, CIUDA...",6720,No Address,No C.P.,GOVERNMENT,Government,NO
334,801950,OTHER GOVERNMENT DISTRIBUTORS,INSTITUTO NACIONAL DE ENFERME,INSTITUTO NACIONAL DE ENFERMEDADES RESPIRATORI...,ISSSTE,NO LEIDO,SOLD TO,801950,-,1,...,7,24,45862,"CALZ DE TLALPAN NO 4502COL SECCION XVI, TLALPA...",14080,No Address,No C.P.,GOVERNMENT,Government,NO
335,802050,OTHER GOVERNMENT DISTRIBUTORS,INSTITUTO NACIONAL DE CIENCIAS,INSTITUTO NACIONAL DE CIENCIAS MEDICAS Y NUTRI...,ISSSTE,NO LEIDO,SOLD TO,802050,-,1,...,7,24,45862,AV VASCO DE QUIROGA NO 15 COLBELISARIO DOMINGU...,14080,No Address,No C.P.,GOVERNMENT,Government,NO
336,903341,OTHER GOVERNMENT DISTRIBUTORS,SERVICIOS Y DISTRIBUCION ALTA ESPECIALIDAD HOS...,INSTITUTO NACIONAL DE CIENCIAS MEDICAS Y NUTRI...,ISSSTE,NO LEIDO,SOLD TO,903341,-,-,...,8,20,45889,No Address,0,No Address,No C.P.,GOVERNMENT,Government,NO


Normalización de columnas y segmentación.

In [45]:
clientes = clientes_raw[['Codigo Cliente', 'Customer_Description', 'CAT MAESTRO VIEJO']]
clientes = clientes.rename(columns={
    "Codigo Cliente": "id_cliente",
    "Customer_Description": "nombre_cliente",
    "CAT MAESTRO VIEJO": "categoría"
})

Se empleó un modelo LLM para clasificar los nombres de los clientes entre personas naturales y empresas, con el objetivo de mejorar la imputación de valores nulos en la columna categoría, tomando en cuenta dicho criterio, para ello se implementó una sencilla funcion.

In [46]:
empresas = [
    'TRANSPORTISTAS NUTRICIONALES', 'ABBOTT LABORATORIES DE MEXICO',
    'CASA MARZAM', 'FARMACIA GUADALAJARA', 'ALMACEN DE DROGAS', 'NADRO',
    'FARMACOS NACIONALES', 'NUEVA WAL MART DE MEXICO',
    'SERVICIOS EN PUERTOS Y TERMINA', 'SOCIEDAD DE BENEFICENCIA ESPA',
    'FARMACIA PARIS S.A. DE C.V.', 'DISTRIBUIDORA DEKAFARMA',
    'FARMACOS ESPECIALIZADOS', 'MEDICA SUR',
    'DISTRIBUCION ESPECIALIZADA DE', 'RAMA FARMACEUTICA',
    'REACTIVOS Y QUIMICOS SA DE CV', 'COSTCO DE MEXICO',
    'TIENDAS COMERCIAL MEXICANA SA', 'TIENDAS CHEDRAUI',
    'SUPERMERCADOS INTERNACIONALES', 'SERVICIO COMERCIAL GARIS SA DE',
    'FARMACIAS BENAVIDES', 'TIENDAS SORIANA', 'TRANSPORTISTAS FARMA',
    'VENTA PUBLICO GENERAL ANI', 'VENTAS DE ACTIVO A EMPLEADOS',
    'VENTA DE ACTIVO A TERCEROS', 'OPERADORA DE HOSPITALES',
    'GERENCIA MEDICA ANI', 'GRUPO FARMACOS ESPECIALIZADOS',
    'REBATE ANI', 'ESPECIALISTAS EN NUTRICION TOT',
    'FARMACIA ALIANZA', 'NUTRAN, S.A. DE C.V.',
    'WWPL MEXICO, S.A. DE C.V.', 'SOLUCIONES E INOVACIONES INTEG',
    'VACANTE', 'JUST SERVICE LOGISTICA Y COMER',
    'PROYECTOS LOGISTICOS JIRAMA SA', 'TRANSPORTES REFRIGERADOS MARMA',
    'CARGA Y EXPRESS SA DE CV', 'SUMINISTROS MEDICOS DEL CENTRO',
    'FUNDACION TELETON MEXICO A.C.', 'I.S.S.S.T.E.', 'IMSS UMF 47',
    'DISTRIBUIDORA DISUR SA DE CV', 'IMSS HOSP GRAL ZONA 1',
    'GENOMI-K SA DE CV', 'VOLUNTARIAS DEL HOSPITAL DEL',
    'MEDICAL CORPORATION GROUP', 'OPERADORA E IMPULSORA LOGISTIC',
    'INSTITUTO DE SEGURIDAD SOCIAL', 'CASA DE LA AMISTAD PARA NI#OS',
    'ASESORIA Y ADMINISTRACION DE', 'ABBOTT NUTRITION LTD',
    'CARITAS DE JALAPA AC', 'REX FARMA',
    'GALUMEDICAL SA DE CV', 'ABBOTT HEALTHCARE COSTA RICA S',
    'ABBOTT LABORATORIES SERVICES', 'VENDING MACHINE ANI',
    'ARTICULOS MEDICOS Y HOSPITALAR', 'DEFA, S.A. DE C.V.',
    'NUTRIMENTOS INTELIGENTES S.A.', 'BENMATIC S.A. DE C.V.',
    'ABBOTT LABORATORIES-SAUDI', 'TRANSPORTES PITIC S.A. DE C.V.',
    'TRANSPORTES DE LA TORRE', 'FARMACIAS EL FENIX DEL CENTRO',
    'ABBOTT LABORATORIES PUERTO', 'TKM CUSTOMER SOLUTIONS',
    'FARMACIAS DE MAZATLAN SA DE CV', 'COMERCIAL CITY FRESKO',
    'ENLACE Y DISTRIBUCION FARMACEU', 'FARMACIAS MODERNAS DE TIJUANA',
    'FARMACIAS DE SIMILARES', 'MAS BODEGA Y LOGISTICA',
    'MEDICAL LOGISTIC SA DE CV', 'FARMACON', 'OPERADORA FUTURAMA',
    'CADENA COMERCIAL OXXO SA DE CV', 'DISTRIBUIDORA LACTYMEDIC',
    'SERVICIOS COMERCIALES AMAZON', 'ELEMENT FLEET MANAGEMENT CORPO',
    'DISTRIBUCIONES NUTRICIONALES', 'VENTA PUBLICO GENERAL ANI DHL',
    'FARMATALAM DE MEXICO', 'ASTRAL DEL PACIFICO',
    'REVOLUCION RADICAL', 'ABBOTT LABORATORIES ARGENTINA',
    'DHL METROPOLITAN LOGISTICS SC', 'FARMACEUTICOS MAYPO',
    'CENTRAL DETALLISTA', 'PRODUCTOS NUTRICION RENAL',
    'INSTITUTO NACIONAL DE PEDIATRI', 'OM NUTRICION', 'MAXIVA SA DE CV',
    'PROQUIGAMA', 'RALCA', 'COMERCIALIZADORA PENTAMED',
    'DISTRIBUIDORA INTERNACIONAL DE', 'MEDIGROUP DEL PACIFICO',
    'DISTRIBUIDORA DE FARMACOS Y FR', 'HISA FARMACEUTICA',
    'SOLUCIONES BRIO', 'VIDA MEDICAL', 'DEREMATE.COM DE MEXICO',
    'MEDIKAMENT DE MEXICO SA DE CV', 'ABBOTT IRELAND',
    'MARCRAN DISTRIBUCIONES', 'CREATIVE GADGETS SA DE CV',
    'NUTRIMEDIC HOGAR', 'COMERCIALIZADORA MEDICA GV',
    'MACONDGMX', 'GRADO QUIRURGICO', 'PHARMA PLUS',
    'VENTA PUBLICO GENERAL ANI ECOM', 'DISTRIBUIDORA Y COMERCIALIZADO',
    'ABBOTT NUTRITION', "MOEDANO'S OTC", 'SANVITE',
    'FARMACOS DAROVI', 'SMED & HEALTH', 'CAMSA DE OCCIDENTE',
    'PHARMA TYCSA', 'CORPORATIVO FARMACEUTICO',
    'DISTRIBUIDORA MEDICA CODIMED', 'INSTITUTO DE SEGURIDAD Y',
    'IMPULSO FARMACEUTICO DE OCCIDE', 'SERVICIOS DE SALUD DEL INSTITU',
    'HOSPITAL JUAREZ DE MEXICO', 'HOSPITAL GENERAL DE MEXICO',
    'INSTITUTO NACIONAL DE ENFERME', 'INSTITUTO NACIONAL DE CIENCIAS',
    'SERVICIOS Y DISTRIBUCION ALTA ESPECIALIDAD HOSPITALARIA S DE RL DE CV',
    'COMERCIALIZADORA MEDMONT SA DE CV'
]


In [47]:
personas = [
    'ZAVALA SALAZAR LAURA ELENA',
    'ORTIZ BERNAL ANA DANIELA',
    'AGUILAR REYES ALEJANDRO',
    'RODRIGUEZ GARCIA NORMA EDITH',
    'CHAVEZ FONSECA MARIANA',
    'ROSAS COLCHADO RUBEN',
    'SANCHEZ HERNANDEZ VERONICA',
    'FELIPE WENZEL CANTO',
    'TORRES ZAPATA SARA IRENE',
    'MARTINEZ BOYER JOAQUIN RICARDO',
    'LEONARDO ESTEBAN CHAVEZ ALDAMA',
    'MARIA ELENA MARIN MENDOZA',
    'LUCIA SIERRA ROBLES',
    'JOSE DE JESUS MUNOZ NAVARRO',
    'SANCHEZ CHAVEZ BRENDA MAGDALEN',
    'FELIX ACEVES SEPULVEDA',
    'JOSE LUIS GARCIA NEGRETE',
    'SILVIA CAROLINA RUIZ FLORES',
    'ESTRADA RAMOS LEONARDA',
    'IRMA FABIOLA VALDEZ GONZALEZ',
    'ROSA NELLY AGUILAR HERNANDEZ',
    'SONIA FLORES OSEQUERA',
    'FERNANDEZ ESTRADA CAROLINA ILE',
    'FRANCISCO JAVIER ESCALONA OCHO',
    'ROMERO JARDINES FELIPE ARMANDO',
    'IVAN ARTURO CRODA VILLANUEVA',
    'ESTRELLA BRENDA DEL RIO ALVARE',
    'SILVIA GARCIA ESCALANTE',
    'ESTEBAN MORA SAUL',
    'MAQUEO BECERRIL MARTHA ADRIANA',
    'SOLANO AGUILAR FERNANDO',
    'PADILLA TORRES ANA LILIA',
    'LOPEZ ESTRADA YOLANDA TERESA',
    'DELGADO MORALES PAMELA ALEJAND',
    'DIAZ QUINTANILLA JESUS EDGAR',
    'FERNANDEZ MUNOZ ELSA LETICIA',
    'REYES BANDA NANCY LIZETH',
    'DANIELA OLIVARES RUIZ',
    'SALVADOR FERNANDO RODRIGUEZ',
    'TULIA GONZALEZ MU#OZ',
    'BERNARDO MONTANO MARTHA LAURA',
    'GARAY BECERRIL OSCAR ARTURO',
    'ALONSO SUAREZ PEDRO',
    'PACHECO MARIA DE JESUS',
    'ROJAS CARRASCO FRANCISCO',
    'MELLADO ELIAS RAUL',
    'HERRERA ROCHA ARACELY ELIZABET',
    'MORALES AMADOR JESUS',
    'MACIEL SALCEDO LIZETTE',
    'ELIZALDE ZALVIDEA QUETZAL',
    'LOPEZ VALENZUELA GUADALUPE',
    'FAJARDO ALVARADO RUBEN ANDRES',
    'TREVINO DUARTE MARIA ESTHER',
    'ORIVE BELLINGER ARMANDO',
    'BALESTERINI ROMERO GABRIELA',
    'PINZON BARBOSA SERGIO',
    'ANGEL MARQUEZ ANA MARIA',
    'TREJO HERNANDEZ ERIKA',
    'FLORES DE LA CRUZ CARMELA',
    'MIJANGOS YEPEZ CAROLINA',
    'MARTINEZ DAGIO LUIS FIDEL',
    'QUINTANA SANTIAGO OSCAR',
    'JUAN PABLO HERNANDEZ GONZALEZ',
    'ANA CRISTINA ANCONA VADILLO'
]


Función de imputación

In [48]:
def impute_nans(df, column, empresas, personas):
    """
    Rellena valores nulos en `column` según el tipo de cliente.
    - Si está en empresas -> "desconocido"
    - Si está en personas -> "persona natural"
    """
    df[column] = df.apply(
        lambda row: (
            "desconocido" if (pd.isna(row[column]) and row["nombre_cliente"] in empresas)
            else "persona natural" if (pd.isna(row[column]) and row["nombre_cliente"] in personas)
            else row[column]
        ),
        axis=1
    )
    return df
    

Se obtiene un dataframe de clientes organizado, sin valores nulos y optimizado para su análisis, así como para su posterior integración en todo el proceso de BI.

In [49]:
clientes = impute_nans(clientes, 'categoría', empresas, personas)
clientes["id_cliente"] = clientes["id_cliente"].astype(str) # Se garantiza que el id_cliente sea string
clientes.info()
clientes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 338 entries, 0 to 337
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id_cliente      338 non-null    object
 1   nombre_cliente  338 non-null    object
 2   categoría       337 non-null    object
dtypes: object(3)
memory usage: 8.0+ KB


,id_cliente,nombre_cliente,categoría
0,906600,TRANSPORTISTAS NUTRICIONALES,desconocido
1,909009,ABBOTT LABORATORIES DE MEXICO,desconocido
2,400043,ZAVALA SALAZAR LAURA ELENA,persona natural
3,400047,ORTIZ BERNAL ANA DANIELA,persona natural
4,400050,AGUILAR REYES ALEJANDRO,persona natural
...,...,...,...
333,801850,HOSPITAL GENERAL DE MEXICO,ISSSTE
334,801950,INSTITUTO NACIONAL DE ENFERME,ISSSTE
335,802050,INSTITUTO NACIONAL DE CIENCIAS,ISSSTE
336,903341,SERVICIOS Y DISTRIBUCION ALTA ESPECIALIDAD HOS...,ISSSTE


In [50]:
clientes.duplicated(subset=['id_cliente']).sum() # se verifica que no haya ids duplicados

0

## **2. Maestro de Materiales.xlsx**
Columnas sugeridas:
- ID Material
- Nombre Material
- Categoría / Familia de productos

Se cargan los datos crudos.

In [51]:
materiales_raw

,CONC1,Local_Code,Codigo de Material,STANDARD DESCRIPTION SUCESSOR,STATUS,D.5.6,D56,1-6-3-3,1633,BPCS SKU DESCRIPTION,...,FLAVOR,FORM_LOCAL,PACK_LOCAL,GLOBAL_PACK,LOCAL PACK GROUP,ORDER S&OP PACK,BRAND S&OP PACK,BRAND 5 FACTORS,CATEGORY S&OP PACK,SUBBRAND
0,00S8460140,100S846132140,100S846132140,ENSURE ADV CHOCO 400G,1-VIGENTE,1-00S846-132-140-0140,100S846132140140,1-00S846-132-140,100S846132140,ENSURE ADV HMB CHOCOLATE 400G,...,CHOCOLATE,POWDER,400G,400 GM,NaN,8,ENSURE,ENSURE,TOTAL ADULT,ENS_ADV
1,00S8470140,100S847132140,100S847132140,ENSURE ADV FRESA 400G,1-VIGENTE,1-00S847-132-140-0140,100S847132140140,1-00S847-132-140,100S847132140,ENSURE ADV HMB FRESA 400G,...,FRESA,POWDER,400G,400 GM,NaN,8,ENSURE,ENSURE,TOTAL ADULT,ENS_ADV
2,679800126,1067980066126,1067980066126,ENSURE ADV RPB CAFE,1-VIGENTE,1-067980-066-066-0126,1067980066066120,1-067980-066-066,1067980066066,ENSURE ADVANCE CAFE 237ML,...,CAFÉ,LIQUID,237ML,237 ML (8 OZ),NaN,8,ENSURE,ENSURE,TOTAL ADULT,ENS_ADV
3,676580126,1067658066126,1067658066126,ENSURE ADV RPB CHOCO,1-VIGENTE,1-067658-066-066-0126,1067658066066120,1-067658-066-066,1067658066066,ENSURE ADVANCE CHOCO 237ML,...,CHOCOLATE,LIQUID,237ML,237 ML (8 OZ),NaN,8,ENSURE,ENSURE,TOTAL ADULT,ENS_ADV
4,676710899,1067671066899,1067671066899,ENSURE ADV RPB CHOCO 16PACK,1-VIGENTE,1-067671-066-066-0899,1067671066066890,1-067671-066-066,1067671066066,ENSURE ADVANCE CHOCO 16PACK,...,CHOCOLATE,LIQUID,16PACK,16x8 OZ,NaN,8,ENSURE,ENSURE,TOTAL ADULT,ENS_ADV
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1375,07700A0001,107700A001001,107700A001001,REVENUE,6-DUMMY,1-07700A-001-001-0001,107700A001001001,1-07700A-001-001,107700A001001,REVENUE RECOGNITION ANI,...,NATURAL,EACH,REVENUE,1 UNIT,NaN,10,OTHER ADULT,OTHERS,TOTAL ADULT,MEDICAL
1376,00T1170001,100T117001001,100T117001001,REVENUE,6-DUMMY,1-00T117-001-001-0001,100T117001001001,1-00T117-001-001,100T117001001,PEDIALYTE DUMMY,...,NATURAL,LIQUID,REVENUE,1 ML,NaN,7,PEDIALYTE,PEDIALYTE,TOTAL PEDIATRIC,PEDIALYTE
1377,675510001,1067551001001,1067551001001,REVENUE,6-DUMMY,1-067551-001-001-0001,1067551001001000,1-067551-001-001,1067551001001,PEDIASURE DUMMY,...,NATURAL,LIQUID,REVENUE,1 ML,NaN,6,PEDIASURE,PEDIASURE,TOTAL PEDIATRIC,PED_BASE
1378,00700A0001,100700AR01001,100700AR01001,REVENUE,6-DUMMY,1-00700A-001-001-0001,100700A001001001,1-00700A-R01-001,100700AR01001,REBATE OTHER PEDIATRIC NUTR,...,NATURAL,EACH,REVENUE,1 UNIT,NaN,1,OTHER IMF,IMF,TOTAL PEDIATRIC,SIMILAC SENSITIVE


Se realizó la normalización de columnas y la segmentación del dataframe de materiales. Este presentaba valores duplicados en la columna id_material, lo cual afectaba la integridad del modelo al no tratarse de una clave única. Para garantizar esta condición, se eliminaron los registros duplicados conservando la estructura correcta.

In [52]:
materiales = materiales_raw[['Codigo de Material', 'STANDARD DESCRIPTION SUCESSOR', 'CATEGORY MODEL']]
materiales = materiales.rename(columns={
    "Codigo de Material": "id_material",
    "STANDARD DESCRIPTION SUCESSOR": "nombre_material",
    "CATEGORY MODEL" : "categoría"
})

materiales["id_material"] = materiales["id_material"].astype(str)   # Se garantiza que el id_material sea string 
materiales.drop_duplicates(subset=['id_material'], keep='first', inplace=True) # Se eliminan los duplicados en id_material
materiales.info()
materiales

<class 'pandas.core.frame.DataFrame'>
Index: 367 entries, 0 to 1379
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id_material      367 non-null    object
 1   nombre_material  367 non-null    object
 2   categoría        367 non-null    object
dtypes: object(3)
memory usage: 11.5+ KB


,id_material,nombre_material,categoría
0,100S846132140,ENSURE ADV CHOCO 400G,ADULT NUTRITIONALS
1,100S847132140,ENSURE ADV FRESA 400G,ADULT NUTRITIONALS
2,1067980066126,ENSURE ADV RPB CAFE,ADULT NUTRITIONALS
3,1067658066126,ENSURE ADV RPB CHOCO,ADULT NUTRITIONALS
4,1067671066899,ENSURE ADV RPB CHOCO 16PACK,ADULT NUTRITIONALS
...,...,...,...
1375,107700A001001,REVENUE,ADULT NUTRITIONALS
1376,100T117001001,REVENUE,HYDRATION
1377,1067551001001,REVENUE,PEDIATRIC NUTRITION
1378,100700AR01001,REVENUE,PEDIATRIC NUTRITION


## **3. In_Market.xlsx**
Columnas:
- Periodo
- Cliente
- Material
- Unidades
- Valor

Se cargan los datos crudos.

In [53]:
in_market_raw

,TYPE,VERSION,IMS TYPE,PERIOD,Codigo Cliente,SALES TYPE,TRANSACTION TYPE,Codigo Material,UNITS,Valores
0,In Market,ACTUALS,Read,45292.0,806910.0,Vta Nacional,OFFLINE,1067660066126,110983.0,4.373371e+06
1,In Market,ACTUALS,Read,45292.0,806910.0,Vta Nacional,OFFLINE,1067679066126,118322.0,4.743942e+06
2,In Market,ACTUALS,Read,45292.0,806910.0,Vta Nacional,OFFLINE,1067676066126,95878.0,3.762524e+06
3,In Market,ACTUALS,Read,45292.0,806910.0,Vta Nacional,OFFLINE,1062777066126,63587.0,2.449082e+06
4,In Market,ACTUALS,Read,45292.0,806910.0,Vta Nacional,OFFLINE,1062778066126,51647.0,8.473475e+05
...,...,...,...,...,...,...,...,...,...,...
915603,In Market,ACTUALS,Read,45658.0,821000.0,NaN,NaN,100S482232140,0.0,0.000000e+00
915604,In Market,ACTUALS,In=To,45658.0,101010.0,NaN,NaN,100S482232140,0.0,0.000000e+00
915605,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
915606,In Market,ACTUALS,Read,45658.0,101010.0,NaN,NaN,1067491066140,0.0,0.000000e+00


Se normalizó y depuró el dataframe  `in_market` mediante la selección y renombrado de columnas, la conversión del campo periodo a formato de fecha, la estandarización de los identificadores de cliente y material como texto, y la eliminación de registros nulos, obteniendo así un conjunto de datos consistente y listo para su análisis.

In [54]:
in_market = in_market_raw[['PERIOD', 'Codigo Cliente', 'Codigo Material', 'UNITS', 'Valores', 'TYPE']]
in_market = in_market.rename(columns={
    'PERIOD': "periodo", 
    'Codigo Cliente': 'id_cliente', 
    'Codigo Material': 'id_material', 
    'UNITS': 'unidades', 
    'Valores': 'valores',
    'TYPE': 'type'
})

in_market['periodo'] = pd.to_datetime(
    in_market['periodo'], 
    unit='d', 
    origin='1899-12-30'
)

in_market["id_material"] = in_market["id_material"].astype(str)        
in_market["id_cliente"] = in_market["id_cliente"].astype("Int64").astype(str) 
in_market.dropna(inplace=True) # Se eliminan los registros nulos ya que el Excel oroginal tiene muchas filas vacias que no aportan informacion
in_market.info()
in_market

<class 'pandas.core.frame.DataFrame'>
Index: 25559 entries, 0 to 915607
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   periodo      25559 non-null  datetime64[ns]
 1   id_cliente   25559 non-null  object        
 2   id_material  25559 non-null  object        
 3   unidades     25559 non-null  float64       
 4   valores      25559 non-null  float64       
 5   type         25559 non-null  object        
dtypes: datetime64[ns](1), float64(2), object(3)
memory usage: 1.4+ MB


,periodo,id_cliente,id_material,unidades,valores,type
0,2024-01-01,806910,1067660066126,110983.0,4.373371e+06,In Market
1,2024-01-01,806910,1067679066126,118322.0,4.743942e+06,In Market
2,2024-01-01,806910,1067676066126,95878.0,3.762524e+06,In Market
3,2024-01-01,806910,1062777066126,63587.0,2.449082e+06,In Market
4,2024-01-01,806910,1062778066126,51647.0,8.473475e+05,In Market
...,...,...,...,...,...,...
915602,2025-01-01,821000,1051112054001,0.0,0.000000e+00,In Market
915603,2025-01-01,821000,100S482232140,0.0,0.000000e+00,In Market
915604,2025-01-01,101010,100S482232140,0.0,0.000000e+00,In Market
915606,2025-01-01,101010,1067491066140,0.0,0.000000e+00,In Market


## **4. To_Market.xlsx**
Columnas:
- Periodo
- Cliente
- Material
- Unidades
- Valor

Se cargan los datos crudos.

In [55]:
to_market_raw

,TYPE,VERSION,IMS TYPE,PERIOD,Codigo de Cliente,Codigo de Material,UNITS,Valores
0,To Market,ACTUALS,Read,2019-01-01,903236,1000079054125,0.0000,0.00
1,To Market,ACTUALS,Read,2019-02-01,903236,1000079054125,0.0000,0.00
2,To Market,ACTUALS,Read,2019-03-01,903236,1000079054125,0.0000,0.00
3,To Market,ACTUALS,Read,2019-04-01,903236,1000079054125,2.0000,2440.00
4,To Market,ACTUALS,Read,2019-05-01,903236,1000079054125,0.0000,0.00
...,...,...,...,...,...,...,...,...
176251,To Market,ACTUALS,Read,2025-07-01,807006,100Y619E32780,3000.0000,2263636.50
176252,To Market,ACTUALS,Read,2025-07-01,821000,1053994066126,0.0001,-2869.24
176253,To Market,ACTUALS,Read,2025-07-01,819200,1053994066126,0.0001,-2665.04
176254,To Market,ACTUALS,Read,2025-07-01,819200,1062774066126,0.0001,-7292.60


Se normalizó el dataframe `to_market` mediante la selección y renombrado de columnas, la estandarización de los identificadores de cliente y material como texto y la eliminación de registros nulos, obteniendo un conjunto de datos limpio y estructurado para su análisis.

In [56]:
to_market = to_market_raw[['PERIOD', 'Codigo de Cliente', 'Codigo de Material', 'UNITS', 'Valores', 'TYPE']]
to_market = to_market.rename(columns={
    'PERIOD': "periodo", 
    'Codigo de Cliente': 'id_cliente', 
    'Codigo de Material': 'id_material', 
    'UNITS': 'unidades', 
    'Valores': 'valores',
    'TYPE': 'type'
})


to_market["id_material"] = to_market["id_material"].astype(str)        
to_market["id_cliente"] = to_market["id_cliente"].astype(str)   
to_market.dropna(inplace=True)
to_market.info()
to_market

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176256 entries, 0 to 176255
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   periodo      176256 non-null  datetime64[ns]
 1   id_cliente   176256 non-null  object        
 2   id_material  176256 non-null  object        
 3   unidades     176256 non-null  float64       
 4   valores      176256 non-null  float64       
 5   type         176256 non-null  object        
dtypes: datetime64[ns](1), float64(2), object(3)
memory usage: 8.1+ MB


,periodo,id_cliente,id_material,unidades,valores,type
0,2019-01-01,903236,1000079054125,0.0000,0.00,To Market
1,2019-02-01,903236,1000079054125,0.0000,0.00,To Market
2,2019-03-01,903236,1000079054125,0.0000,0.00,To Market
3,2019-04-01,903236,1000079054125,2.0000,2440.00,To Market
4,2019-05-01,903236,1000079054125,0.0000,0.00,To Market
...,...,...,...,...,...,...
176251,2025-07-01,807006,100Y619E32780,3000.0000,2263636.50,To Market
176252,2025-07-01,821000,1053994066126,0.0001,-2869.24,To Market
176253,2025-07-01,819200,1053994066126,0.0001,-2665.04,To Market
176254,2025-07-01,819200,1062774066126,0.0001,-7292.60,To Market


Se unificaron los dataframes in_market y to_market en un único conjunto denominado ventas, el cual, junto con las tablas de clientes y materiales, fue exportado en formato CSV a la carpeta cleaned, garantizando así una versión depurada y lista para su análisis.

In [59]:
ventas = pd.concat([in_market, to_market], ignore_index=True)

ventas.to_excel("../datasets/cleaned2/ventas.xlsx", index=False)
clientes.to_excel("../datasets/cleaned2/clientes.xlsx", index=False)
materiales.to_excel("../datasets/cleaned2/materiales.xlsx", index=False)